### DETR model training ###
Build the model training script

In [47]:
import sys
import os
import numpy as np
import pandas as pd
import datetime
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import patches

# PyTorch
import torch

# Hugging Face Library
from transformers import RTDetrV2ForObjectDetection, RTDetrImageProcessor
from transformers import TrainingArguments, Trainer

%load_ext autoreload
%autoreload 2
import computervision
from computervision.imageproc import is_image
from computervision.datasets import DTRdataset, get_gpu_info

print(f'Project version: {computervision.__version__}')
print(f'Python version:  {sys.version}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project version: v0.0.1
Python version:  3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]


In [48]:
# Set training device
device, device_str = get_gpu_info()
# Save the date in a string
date_str = datetime.date.today().strftime('%y%m%d')
print(f'Date: {date_str}')

CUDA available: True
Number of GPUs found:  1
Current device ID: 0
GPU device name:   NVIDIA GeForce RTX 3070 Laptop GPU
PyTorch version:   2.8.0a0+34c6371d24.nv25.08
CUDA version:      13.0
CUDNN version:     91200
Date: 250922


In [60]:
# Location of the data
data_dir = os.path.join(os.environ.get('HOME'), 'data')

# Image directories
train_image_dir = os.path.join(data_dir, 'dentex', 'cropped')
val_image_dir = os.path.join(train_image_dir, 'test')

# Model checkpoint and log files
model_dir = os.path.join(data_dir, 'model')
log_dir = os.path.join(model_dir, 'log')
Path(log_dir).mkdir(parents=True, exist_ok=True)

# Data frames with the file names and annotations
train_annotation_file_name = 'train_quadrant_enumeration_dset.parquet'
train_annotation_file = os.path.join(train_image_dir, train_annotation_file_name)
train_df = pd.read_parquet(train_annotation_file)
# Filter the training images
train_df = train_df.loc[train_df['dset'] == 'train']

val_annotation_file_name = 'train_quadrant_enumeration_test_set.parquet'
val_annotation_file = os.path.join(val_image_dir, val_annotation_file_name)
val_df = pd.read_parquet(val_annotation_file)
# Filter the validation images and quadrants and take only the first augmentation
val_df = val_df.loc[
    (val_df['dset'] == 'val') & 
    (val_df['quadrants'].isin([14, 23])) & 
    (val_df['transformation'] == 0)]

# Column names (tooth positions are stored in the ada colum)
label_col = 'label'
label_name_col = 'ada'
file_name_col = 'file_name'
bbox_col = 'bbox'

# Check the images on disk
train_file_list = list(train_df[file_name_col].unique())
train_checked = np.sum([is_image(os.path.join(train_image_dir, file)) for file in train_file_list])
print(f'Images in training data:         {len(file_name_list)}')
print(f'Files checked in training data:  {train_checked}')
print(f'Annotations in training data:    {train_df.shape[0]}')

val_file_list = list(val_df[file_name_col].unique())
val_checked = np.sum([is_image(os.path.join(val_image_dir, file)) for file in val_file_list])
print(f'Images in validation data:       {len(val_file_list)}')
print(f'Files checked in val data:{val_checked}')
print(f'Annotations in validation data:  

Files in training data: 2391
Files checked in training data:   2391
Files in training data: 32
Files checked in training data:   32


In [59]:
val_df.head()

,bbox,quadrant,ada,file_name,file_base_name,quadrants,height,width,transformation,transformation_name,dset
125,"[459, 0, 11, 323]",1,8,val_train_143_14_00.png,train_143,14,471,516,0,test_set,val
126,"[399, 8, 70, 294]",1,7,val_train_143_14_00.png,train_143,14,471,516,0,test_set,val
127,"[313, 0, 99, 314]",1,6,val_train_143_14_00.png,train_143,14,471,516,0,test_set,val
128,"[253, 11, 102, 304]",1,5,val_train_143_14_00.png,train_143,14,471,516,0,test_set,val
129,"[195, 0, 103, 309]",1,4,val_train_143_14_00.png,train_143,14,471,516,0,test_set,val


In [50]:
# Create the label ids (tooth position, but starting from 0)
# The model needs label ids, not labels. So we need to add a label id column

label_name_list = sorted(list(df[label_name_col].unique()))
id2label = dict(zip(range(len(label_list)), label_list))
id2label = {int(label_id): str(label_name) for label_id, label_name in id2label.items()}
label2id = {str(label_name): int(label_id) for label_id, label_name in id2label.items()}

# Add the label column to the data frame
df = df.assign(label=df[label_name_col].apply(lambda pos: label2id.get(str(pos))))
display(df.head(2))
print(df.shape)

,bbox,segmentation,height,width,file_name,file_base_name,quadrants,quadrant,pos,fdi,ada,dset,label
0,"[666, 102, 103, 376]","[[757, 478, 769, 102, 678, 113, 666, 469]]",494,1473,train_0_12.png,train_0,12,1,1,11,8,train,7
1,"[593, 107, 85, 377]","[[666, 484, 678, 110, 607, 107, 604, 299, 619,...",494,1473,train_0_12.png,train_0,12,1,2,12,7,train,6


(36190, 13)


In [ ]:
# Create the data sets